<center><h1> Convolutional Neural Network - TensorFlow </h1><center>

### Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

### Load Data

In [2]:
#from tensorflow.examples.tutorials.mnist import input_data
#data = input_data.read_data_sets('data/MNIST/', one_hot=True)

In [3]:
#print(data)

#print("Size of:")
#print("- Training-set:\t\t{}".format(len(data.train.labels)))
#print("- Test-set:\t\t{}".format(len(data.test.labels)))
#print("- Validation-set:\t{}".format(len(data.validation.labels)))

### Placeholder variables

In [4]:
# Placeholder variable for the input images
x = tf.placeholder(tf.float32, shape=[None, 224*224], name='X')
# Reshape it into [num_images, img_height, img_width, num_channels]
x_image = tf.reshape(x, [-1, 224, 224, 1])

# Placeholder variable for the true labels associated with the images
y_true = tf.placeholder(tf.float32, shape=[None, 88], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


### Function for creating a new Convolution Layer

In [5]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, name):
    
    with tf.variable_scope(name) as scope:
        # Shape of the filter-weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

        # Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))

        # TensorFlow operation for convolution
        layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')

        # Add the biases to the results of the convolution.
        layer += biases
        
        return layer, weights

### Function for creating a new Pooling Layer

In [6]:
def new_pool_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.max_pool(value=input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        return layer

### Function for creating a new ReLU Layer

In [7]:
def new_relu_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.relu(input)
        
        return layer

### Function for creating a new Fully connected Layer

In [8]:
def new_fc_layer(input, num_inputs, num_outputs, name):
    
    with tf.variable_scope(name) as scope:

        # Create new weights and biases.
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
        
        # Multiply the input and weights, and then add the bias-values.
        layer = tf.matmul(input, weights) + biases
        
        return layer

### Create Convolutional Neural Network

In [9]:
# Convolutional Layer 1
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, num_input_channels=1, filter_size=5, num_filters=6, name ="conv1")

# Pooling Layer 1
layer_pool1 = new_pool_layer(layer_conv1, name="pool1")

# RelU layer 1
layer_relu1 = new_relu_layer(layer_pool1, name="relu1")

# Convolutional Layer 2
layer_conv2, weights_conv2 = new_conv_layer(input=layer_relu1, num_input_channels=6, filter_size=5, num_filters=12, name= "conv2")

# Pooling Layer 2
layer_pool2 = new_pool_layer(layer_conv2, name="pool2")

# RelU layer 2
layer_relu2 = new_relu_layer(layer_pool2, name="relu2")

# Convolutional Layer 3
layer_conv3, weights_conv3 = new_conv_layer(input=layer_relu2, num_input_channels=12, filter_size=5, num_filters=24, name= "conv3")

# Pooling Layer 3
layer_pool3 = new_pool_layer(layer_conv3, name="pool3")

# RelU layer 3
layer_relu3 = new_relu_layer(layer_pool3, name="relu3")

# Convolutional Layer 4
layer_conv4, weights_conv4 = new_conv_layer(input=layer_relu3, num_input_channels=24, filter_size=5, num_filters=48, name= "conv4")

# Pooling Layer 4
layer_pool4 = new_pool_layer(layer_conv4, name="pool4")

# RelU layer 4
layer_relu4 = new_relu_layer(layer_pool4, name="relu4")

# Convolutional Layer 5
layer_conv5, weights_conv5 = new_conv_layer(input=layer_relu4, num_input_channels=48, filter_size=5, num_filters=96, name= "conv5")

# Pooling Layer 5
layer_pool5 = new_pool_layer(layer_conv5, name="pool5")

# RelU layer 5
layer_relu5 = new_relu_layer(layer_pool5, name="relu5")

# Flatten Layer
num_features = layer_relu5.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu5, [-1, num_features])

print("Num Features:")
print(num_features)                             
                             
# Fully-Connected Layer 1
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=1000, name="fc1")

# RelU layer 3
layer_relu6 = new_relu_layer(layer_fc1, name="relu6")

# Fully-Connected Layer 2
layer_fc2 = new_fc_layer(input=layer_relu6, num_inputs=1000, num_outputs=88, name="fc2")

Num Features:
4704


In [10]:
# Use Softmax function to normalize the output
with tf.variable_scope("Softmax"):
    y_pred = tf.nn.softmax(layer_fc2)
    y_pred_cls = tf.argmax(y_pred, dimension=1)

### Cost Function

In [11]:
# Use Cross entropy cost function
with tf.name_scope("cross_ent"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Optimizer

In [12]:
# Use Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

### Accuracy

In [13]:
# Accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
# correct_prediction
with tf.name_scope("correct_prediction"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [15]:
# Initialize the FileWriter
writer = tf.summary.FileWriter("Training_FileWriter/")
writer1 = tf.summary.FileWriter("Validation_FileWriter/")

In [16]:
# Add the cost and accuracy to summary
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

In [23]:
num_epochs = 100
batch_size = 88
data_path = 'train.tfrecords'  # address to save the hdf5 file

### TensorFlow Session

In [24]:
#tf.reset_default_graph()
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    
    feature = {'train/image': tf.FixedLenFeature([], tf.string),
               'train/label': tf.FixedLenFeature([], tf.int64)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=num_epochs)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['train/image'], tf.uint8)
    
    # Cast label data into int32
    label = tf.cast(features['train/label'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [224, 224])
    
    print(label)
    
    # Any preprocessing here ...
    
    # Creates batches by randomly shuffling tensors
    images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, capacity=30, num_threads=8, min_after_dequeue=0)
        

    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    sess.run(init_op)
    
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Loop over number of epochs
    for epoch in range(num_epochs):

        start_time = time.time()
        train_accuracy = 0
        
        print("epoch: " + str(epoch))
    
        for batch_index in range(int(88./batch_size)):
            #print("batch " + str(batch_index))
            
            img, lbl = sess.run([images, labels])

            #print(img[0])
            # Get a batch of images and labels
            
            
            #print(img.shape)
            img_right_format = img.reshape((batch_size, -1))
            #print(img_right_format.shape)
            #print(result.shape)
            #pixel_lists = img.reshape(img.shape[:-3] + (-1,3))
            #result = pixel_lists[:, :, 0]
            #print(result.shape)
            
            #print(lbl.shape)
            
            # this is lbl #a = np.array([1, 0, 3])
            b = np.zeros((batch_size, 89))
            b[np.arange(batch_size), lbl] = 1
            b = np.delete(b, -1, axis=1)
            
            #print("b - shape:", b.shape)
            
            #print("b:", b)
            
            
            #print(pixel_lists.shape)
            
            x_batch = img_right_format
            y_true_batch = b
            
            #print(type(x_batch))
            #print(x_batch.shape)
            #print(type(y_true_batch))
            #print(y_true_batch.shape)
            
            # Put the batch into a dict with the proper names for placeholder variables
            feed_dict_train = {x: x_batch, y_true: y_true_batch}
            
            # Run the optimizer using this batch of training data.
            sess.run(optimizer, feed_dict=feed_dict_train)
            
            # Calculate the accuracy on the batch of training data
            train_accuracy += sess.run(accuracy, feed_dict=feed_dict_train)
            print("Train accuracy on batch", round(train_accuracy,5))
            
            # Generate summary with the current batch of data and write to file
            summ = sess.run(merged_summary, feed_dict=feed_dict_train)
            writer.add_summary(summ, epoch*int(88./batch_size) + batch_index)

            # Insert CNN CODE



            # END CNN CODE

            #print("Images " + str(img))
            #print("Labels " + str(lbl))
            #img = img.astype(np.uint8)
            #for j in range(batch_size):
            #    plt.subplot(4, 2, j+1)
            #    plt.imshow(img[j, ...])
            #    plt.title(str(lbl[j]))
            #plt.show()
            
        train_accuracy /= int(88./batch_size)
        
        # Generate summary and validate the model on the entire validation set
        #summ, vali_accuracy = sess.run([merged_summary, accuracy], feed_dict={x:data.validation.images, y_true:data.validation.labels})
        writer1.add_summary(summ, epoch)
        
        end_time = time.time()
        
        print("Epoch "+str(epoch+1)+" completed : Time usage "+str(round(end_time-start_time, 2))+" seconds")
        print("\tAccuracy:")
        print ("\t- Training Accuracy:\t{}".format(round(train_accuracy,2)))
        #print ("\t- Validation Accuracy:\t{}".format(vali_accuracy))
        
    # Stop the threads
    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)
    
    # trying to get a prediction
    
    im = img[0].reshape((1, -1))
    print("corresponding label: ", lbl[0])
    
    feed_dict = {x: im}
    classification = sess.run(y_pred_cls, feed_dict=feed_dict)
    #train_accuracy += sess.run(accuracy, feed_dict=feed_dict)
    print(classification)
    
    
    print("wrote test-dataset stuff into " + str(writer))
    sess.close()
    


Tensor("Cast_5:0", shape=(), dtype=int32)
epoch: 0
Train accuracy on batch 0.01136
Epoch 1 completed : Time usage 0.8 seconds
	Accuracy:
	- Training Accuracy:	0.01
epoch: 1
Train accuracy on batch 0.01136
Epoch 2 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.01
epoch: 2
Train accuracy on batch 0.03409
Epoch 3 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.03
epoch: 3
Train accuracy on batch 0.01136
Epoch 4 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.01
epoch: 4
Train accuracy on batch 0.01136
Epoch 5 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.01
epoch: 5
Train accuracy on batch 0.02273
Epoch 6 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.02
epoch: 6
Train accuracy on batch 0.01136
Epoch 7 completed : Time usage 0.24 seconds
	Accuracy:
	- Training Accuracy:	0.01
epoch: 7
Train accuracy on batch 0.01136
Epoch 8 completed : Time usage 0.25 seconds
	Accuracy:
	-

Train accuracy on batch 0.97727
Epoch 67 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.98
epoch: 67
Train accuracy on batch 0.97727
Epoch 68 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.98
epoch: 68
Train accuracy on batch 0.98864
Epoch 69 completed : Time usage 0.23 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 69
Train accuracy on batch 0.98864
Epoch 70 completed : Time usage 0.23 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 70
Train accuracy on batch 0.98864
Epoch 71 completed : Time usage 0.26 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 71
Train accuracy on batch 0.98864
Epoch 72 completed : Time usage 0.25 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 72
Train accuracy on batch 0.98864
Epoch 73 completed : Time usage 0.23 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 73
Train accuracy on batch 0.98864
Epoch 74 completed : Time usage 0.23 seconds
	Accuracy:
	- Training Accuracy:	0.99
epoch: 74
